In [ ]:
# Figure out what to do about Russia error message

In [19]:
# import os
# import re
import json
import time
import requests
import xlsxwriter
from math import sqrt
from datetime import datetime as dt
from pandas import DataFrame, read_csv, MultiIndex, ExcelWriter

In [ ]:
# This block is for building and sending the API query
# Also fills missing values with 0
def build_query(iso): # Add adm1 as an optional parameter?
    q = (
        "SELECT iso, adm1, alert__year, alert__week, SUM(alert__count) " +
        "as alert__count, confidence__cat " +
        "FROM mytable " +
        f"WHERE iso='{iso}' AND confidence__cat='h' " +
        "GROUP BY iso, adm1, alert__year, alert__week"
    )
    return q


def query_params(q):
    params = {
        'sql':q
    }
    return params


def send_request(base_url, data_id, iso): 
    q = build_query(iso)
    p = query_params(q)
    r = requests.get(base_url + '/' + data_id, params=p)
    
    if r.status_code == 200:
        return r.json()['data']

    else:
        print('Something went wrong...')
        print('Status code: ', r.status_code)
        print('Error message: ', r.json()['errors'][0]['detail'])

In [12]:
# This block is for processing data after query
# Update this to make year range dynamic (based on today)
def create_multiindex(df, names):
    countries = df.iso.unique()
    regions = df.adm1.unique()
    years = range(2012, 2022)
    weeks = range(1, 53)

    mi = MultiIndex.from_product(
        iterables=[countries, regions, years, weeks],
        names=names
    )

    return mi


def fill_missing(df):
    names = ['iso', 'adm1', 'alert__year', 'alert__week']
    mi = create_multiindex(df, names)

    filled = (
        df
            .set_index(names)
            .reindex(mi)
            .reset_index()
            .fillna(0)
    )
    
    return filled

In [13]:
# This block is for calculating significance score
# Clean these up and compartmentalize better
def get_cw_cy():
    today = dt.today().isocalendar()
    # cy = today[0]
    if today[1] == 1:
        cw == 52
        cy == today[0] - 1
    else:
        cw = today[1] - 1
        cy = today[0]

    return cw, cy


def make_lookup(df, adm1):
    df = df[df['adm1']==adm1]
    return df.pivot_table(
        index='alert__year',
        columns='alert__week',
        values='alert__count'
    ).fillna(0)


def vizzuality_sd(lookup, week, mean):
    return sqrt(sum((lookup[week] - mean)**2)/len(lookup[week]))


def significance_variables(lookup):
    # Get current week and year
    week, year = get_cw_cy()
    x = lookup[week][year]
    mu = lookup[week].mean()
    sd = vizzuality_sd(lookup, week, mu)
    return x, mu, sd


def calc_sigscore(df, adm1): # Add number of weeks here
    lookup = make_lookup(df, adm1)
    x, mu, sd = significance_variables(lookup)
    if x - mu == 0:
        significance = 0
    else:
        significance = (x - mu) / sd
    return significance, x


def sigscore_tuple(df, country, region):
    sig_score, current_alerts = calc_sigscore(df, region)
    return country, region, sig_score, current_alerts

In [40]:
# This block is to calculate significance score for each adm1 in all countries
def all_adm1_significance(iso_list, verbose=False):
    BASE_URL = 'https://api.resourcewatch.org/v1/query'
    DATA_ID = '54bb00e8-9888-494a-bcd8-9fd3760fe384'

    final_out = []
    # start = 1
    # end = len(iso_list)
    for country in iso_list:
        # if verbose == 2:
        print(country)
        # if verbose == 1:
        #     print(f'\r{(start/end) * 100:.2f}%', end='', flush=True)
        data = send_request(BASE_URL, DATA_ID, country)
        time.sleep(2)
        if data:
            df = fill_missing(DataFrame(data))
            regions = df.adm1.unique()
            out = [sigscore_tuple(df, country, region) for region in regions]
            final_out += out
        # start += 1
    
    rank_df = DataFrame(final_out, columns=['country', 'adm1', 'significance', 'current_alerts'])
    sorted = rank_df.sort_values(by=['significance'], ascending=False)

    return sorted

In [41]:
# BASE_URL = 'https://api.resourcewatch.org/v1/query'
# DATA_ID = '54bb00e8-9888-494a-bcd8-9fd3760fe384'
# q = build_query('UGA')
# p = query_params(q)
# r = requests.get(BASE_URL + '/' + DATA_ID, params=p)

In [42]:
# r.json()

In [43]:
def gadm_countries(file='gadm_adm1.csv'):
    keep_cols = ['GID_0', 'NAME_0', 'GID_1', 'NAME_1']
    gadm_df = read_csv(file, usecols=keep_cols)
    iso_list = gadm_df.GID_0.unique()
    return iso_list

In [44]:
iso_list = gadm_countries()

In [45]:
# There is a problem with Russia at the moment
# Group by query produces too many results
# Need to reduce the number of rows
# Consider more restrictive "where" clause or less "group by" criteria
start = dt.now()
print(start)
top_adm1 = all_adm1_significance(iso_list)
end = dt.now()
print(f'Total time: {start - end}')

__year, alert__week
LBN
Query:  SELECT iso, adm1, alert__year, alert__week, SUM(alert__count) as alert__count, confidence__cat FROM mytable WHERE iso='LBN' AND confidence__cat='h' GROUP BY iso, adm1, alert__year, alert__week
LBR
Query:  SELECT iso, adm1, alert__year, alert__week, SUM(alert__count) as alert__count, confidence__cat FROM mytable WHERE iso='LBR' AND confidence__cat='h' GROUP BY iso, adm1, alert__year, alert__week
LBY
Query:  SELECT iso, adm1, alert__year, alert__week, SUM(alert__count) as alert__count, confidence__cat FROM mytable WHERE iso='LBY' AND confidence__cat='h' GROUP BY iso, adm1, alert__year, alert__week
LCA
Query:  SELECT iso, adm1, alert__year, alert__week, SUM(alert__count) as alert__count, confidence__cat FROM mytable WHERE iso='LCA' AND confidence__cat='h' GROUP BY iso, adm1, alert__year, alert__week
LIE
Query:  SELECT iso, adm1, alert__year, alert__week, SUM(alert__count) as alert__count, confidence__cat FROM mytable WHERE iso='LIE' AND confidence__cat='h' 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Merge this and the next block into function calls
top_adm1['score'] = top_adm1.significance * top_adm1.current_alerts
top_score = top_adm1.sort_values(['score'], ascending=False).head(20)
top_sig = top_adm1[top_adm1['current_alerts'] > 50].head(20)
print(top_score)
print(top_sig)
# top_adm1.head(5)

In [ ]:
today = dt.today()
filename = 'top_20_output_' + today.strftime('%Y%m%d') + '.xlsx'
# Create pandas excel writer
writer = ExcelWriter(filename, engine='xlsxwriter')
# Write each dataframe to a different worksheet
top_sig.to_excel(writer, sheet_name='top_significance')
top_score.to_excel(writer, sheet_name='top_score')
# Close the pandas excel writer
writer.save()